# Scene Beat to Prose

Train the LLM to write paragraphs in my writing style, based on scene beats.

In [4]:
from mlx_lm import load, generate
from openai import OpenAI
import json
import os
from tqdm import tqdm
from prompts import world_explanation
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def separate_chapters(text):
    chapters = []
    lines = text.split('\n')
    current_chapter = []
    weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
    
    for i, line in enumerate(lines):
        if i < len(lines) - 1 and any(lines[i+1].startswith(day) for day in weekdays):
            if current_chapter:
                chapters.append('\n'.join(current_chapter))
                current_chapter = []
        current_chapter.append(line)
    
    if current_chapter:
        chapters.append('\n'.join(current_chapter))
    
    return chapters

def chunk_chapters(chapters, max_words=500):
    chunked_chapters = []
    for chapter in chapters:
        lines = chapter.split('\n')
        current_chunk = []
        word_count = 0
        
        for line in lines:
            line_words = len(line.split())
            if word_count + line_words > max_words:
                if current_chunk:
                    chunked_chapters.append('\n'.join(current_chunk))
                current_chunk = [line]
                word_count = line_words
            else:
                current_chunk.append(line)
                word_count += line_words
        
        if current_chunk:
            chunked_chapters.append('\n'.join(current_chunk))
    
    return chunked_chapters

def paragraph_to_scene_beats(chunk, model, tokenizer, use_openai=False, openai_model="gpt-4o-mini"):
    prompt = f"""
    Du bist ein professioneller Autor und arbeitest an einer Fantasy-Szene für den Roman "ANSTURM": {world_explanation}
    
    Vor dir liegt ein Textparagraph, und deine Aufgabe ist es, die zugrundeliegenden Scene Beats zu rekonstruieren. Jeder Beat sollte die Motivationen der Charaktere, die äusseren Umstände und das Ziel der Szene zusammenfassen, aber auf einfache und knappe Weise.

    Hier ist ein Beispiel für Scene Beats, wie du sie schreiben sollst:

    - Sokrate rennt verzweifelt durch die Dunkelheit, verfolgt von den Gedanken, dass er zu spät kommen könnte, um das Leben, das er kennt, zu retten. Er ist erschöpft, doch der Drang, die gesammelten Informationen weiterzugeben, treibt ihn an.
    - Der Wald um ihn herum wird immer dunkler, der Mond geht auf, und damit erklingt das Heulen der Werwölfe in der Ferne. Er spürt die Bedrohung näherkommen, kämpft aber mit aller Kraft weiter. Die Schreie aus der Ferne holen ihn in die Realität zurück.
    - Vor dem Portal ergreift er das Amulett. Er weiss, dass er das Portal damit öffnen kann, indem er es hinhält. Kurz danach wird er von einem Werwolf zu Boden gerissen. Schmerz durchfährt seinen Körper. Er erkennt, dass es kein Entkommen mehr gibt.
    - Die Werwölfe umringen ihn, und der Anführer beißt zu. Mit dem letzten Gedanken an sein altes Ich, bevor er vollständig verwandelt wird, akzeptiert Sokrate, dass sein altes Leben endet.

    Der Paragraph:

    "{chunk}"

    Antworte nur mit den Scene Beat(s), ohne weitere Einleitung oder Erklärung.
    """

    try:
        if use_openai:
            response = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model=openai_model,
                max_tokens=1000
            )
            scene_beats = response.choices[0].message.content
        else:
            messages = [{"role": "user", "content": prompt}]
            prompt = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            scene_beats = generate(model, tokenizer, prompt=prompt, verbose=False, max_tokens=1000)
        return scene_beats
    except Exception as e:
        print(f"Error generating rephrased text: {e}")
        return f"EXCEPTION: {e}"

# Function to generate scene beats for chunks
def generate_scene_beats(input_file, output_file):
    # Load existing results if any
    existing_results = {}
    if os.path.exists(output_file):
        with open(output_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                existing_results[data['id']] = data

    # Process chunks and generate scene beats
    with open(input_file, 'r') as in_file, open(output_file, 'a') as out_file:
        for line in tqdm(in_file):
            data = json.loads(line)
            chunk_id = data['id']
            
            # Skip if already processed
            if chunk_id in existing_results:
                continue
            
            scene_beats = paragraph_to_scene_beats(data['chunk'], model, tokenizer, use_openai=True)
            print(f'Generated scene beats for chunk {chunk_id}')
            
            result = {
                "id": chunk_id,
                "chunk": data['chunk'],
                "scene_beats": scene_beats
            }
            
            json.dump(result, out_file)
            out_file.write('\n')
            out_file.flush()  # Ensure data is written immediately

CHUNK_CHAPTERS = False
CHUNK_SIZE = 250
VERSION = "v2"

# Load model and tokenizer
model, tokenizer = load("models/frdm-Llama-3.1-8B-Write")

# Read and process the text
text = open('data/ansturm.txt', 'r').read()
if CHUNK_CHAPTERS:
    chapters = separate_chapters(text)
    prose_paragraphs = chunk_chapters(chapters, max_words=CHUNK_SIZE)
else:
    prose_paragraphs = separate_chapters(text)

# Save chunked chapters to a JSONL file
chunked_file = f'data/chapters_{VERSION}.jsonl'
with open(chunked_file, 'w') as f:
    for i, chunk in enumerate(prose_paragraphs):
        json.dump({"id": i, "chunk": chunk}, f)
        f.write('\n')

# Generate scene beats
output_file = f'data/chapters_with_beats_{VERSION}.jsonl'
generate_scene_beats(chunked_file, output_file)

print("Scene beats generation completed.")

1it [00:02,  2.53s/it]

Generated scene beats for chunk 0


2it [00:06,  3.11s/it]

Generated scene beats for chunk 1


3it [00:08,  2.95s/it]

...

Generated scene beats for chunk 24


26it [01:12,  2.89s/it]

Generated scene beats for chunk 25


27it [01:15,  2.80s/it]

Generated scene beats for chunk 26
Scene beats generation completed.


In [5]:
import json

def create_dataset_entry(scene_beats, paragraph):
    messages = [
        {"role": "system", "content": "Du bist der Fantasy-Autor Yvo K. Jedes Mal, wenn ich dir Scene Beats vorschreibe, schreibst du die komplette Szene auf der Grundlage der Idee. Schliesse die Szene nicht selbst ab, sondern halte dich genau an die Scene Beats."},
        {"role": "user", "content": scene_beats},
        {"role": "assistant", "content": paragraph}
    ]
    return json.dumps({"messages": messages})

def create_dataset(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            data = json.loads(line)
            dataset_entry = create_dataset_entry(data['scene_beats'], data['chunk'])
            outfile.write(dataset_entry + '\n')

# Create the dataset
input_file = f'data/chapters_with_beats_{VERSION}.jsonl'
output_file = f'data/scene_beat_to_prose_dataset_{VERSION}.jsonl'
create_dataset(input_file, output_file)

print("Dataset creation completed.")

Dataset creation completed.


In [1]:
import os
import random

def create_train_valid_test_split(input_file, output_folder, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, seed=42):
    # Ensure ratios sum to 1
    assert abs(train_ratio + valid_ratio + test_ratio - 1.0) < 1e-5, "Ratios must sum to 1"

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Read all lines from the input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Shuffle the lines
    random.seed(seed)
    random.shuffle(lines)

    # Calculate split indices
    train_split = int(len(lines) * train_ratio)
    valid_split = train_split + int(len(lines) * valid_ratio)

    # Split the data
    train_data = lines[:train_split]
    valid_data = lines[train_split:valid_split]
    test_data = lines[valid_split:]

    # Write train data
    train_file = os.path.join(output_folder, 'train.jsonl')
    with open(train_file, 'w') as outfile:
        outfile.writelines(train_data)

    # Write validation data
    valid_file = os.path.join(output_folder, 'valid.jsonl')
    with open(valid_file, 'w') as outfile:
        outfile.writelines(valid_data)

    # Write test data
    test_file = os.path.join(output_folder, 'test.jsonl')
    with open(test_file, 'w') as outfile:
        outfile.writelines(test_data)

    print(f"Train-validation-test split created in {output_folder}")
    print(f"Train samples: {len(train_data)}")
    print(f"Validation samples: {len(valid_data)}")
    print(f"Test samples: {len(test_data)}")

# Usage
dataset_name = f"frdm-Llama-3.1-8B-Write-Beat-to-Prose-{VERSION}"

input_file = f'data/scene_beat_to_prose_dataset_{VERSION}.jsonl'
output_folder = f'data/{dataset_name}'
create_train_valid_test_split(input_file, output_folder)

Train-validation-test split created in data/frdm-Llama-3.1-8B-Write-Beat-to-Prose-v1
Train samples: 190
Validation samples: 23
Test samples: 25


In [14]:
# Model inference
from mlx_lm import load, generate
import json

def generate_scene(model, tokenizer, scene_beats, max_tokens=1024, temp=0.5):
    messages = [
        {"role": "system", "content": "Du bist der Fantasy-Autor Yvo K. Jedes Mal, wenn ich dir einen Scene Beat vorschreibe, schreibst du die komplette Szene auf der Grundlage der Idee. Schliesse die Szene nicht selbst ab, sondern halte dich genau an die Scene Beats. Schliesse nicht mit einer Vorahnung ab."},
        {"role": "user", "content": scene_beats}
    ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    return generate(model, tokenizer, prompt=json.dumps(prompt), max_tokens=max_tokens, temp=temp, verbose=True)

# Load models
raw_model, raw_tokenizer = load("models/frdm-Llama-3.1-8B-Write")
fine_tuned_model, fine_tuned_tokenizer = load("models/frdm-Llama-3.1-8B-Write", adapter_path="models/frdm-Llama-3.1-8B-Write-Beat-to-Prose-v1")

# Scene beats
scene_beats = """
- Kontext: Fantasy Roman ANSTURM
- Charaktere: Raven, Dusk, Caleor, alles Jungs in der 8. Klasse.
- Perspektive: 1. Person Sicht Caleor, männlich, Ich-Erzähler
- Zeitform: Präsens
- Raven übernimmt die Kontrolle und stellt eine zynische Frage, die die Unlogik der Situation hervorhebt. Er ist frustriert und handelt impulsiv, indem er die Schlüssel an sich reißt und die Tür zuschlägt, bevor er entschlossen Richtung Büro des Schulleiters geht.
- Die restliche Gruppe bleibt zurück. Der Protagonist Caleor lenkt die Aufmerksamkeit auf die bevorstehenden Ferien und versucht, die Stimmung aufzuhellen, indem er die anderen nach Plänen fragt.
- Dusk schlägt eine Biketour in die Berge vor, die an einen schönen Ort führt, den sie letztes Jahr entdeckt haben. Die Gruppe beginnt, praktische Details zu planen, um das Abenteuer vorzubereiten.
- Am Montag versammeln sich alle im Garten des Protagonisten, bereit für das Abenteuer. Das Wetter ist kühl, der Herbst kündigt den nahenden Winter an. Sie packen ihre Bikes und starten den Anstieg zu einem erhöhten Platz am See, während sie sich der winterlichen Berglandschaft nähern.
"""

raw_scene = generate_scene(raw_model, raw_tokenizer, scene_beats)
fine_tuned_scene = generate_scene(fine_tuned_model, fine_tuned_tokenizer, scene_beats, temp=0.4)

# Generate scenes
print("Raw model:")
print(raw_scene)
print("\n\n")
print("Fine-tuned model:")
print(fine_tuned_scene)

NotADirectoryError: [Errno 20] Not a directory: 'models/frdm-Llama-3.1-8B-Write-Beat-to-Prose-v1/0000095_adapters.safetensors/adapter_config.json'

In [12]:
fine_tuned_scene = generate_scene(fine_tuned_model, fine_tuned_tokenizer, scene_beats, temp=0.4)

Prompt: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are the fantasy author Yvo K. Each time I prompt you with a scene beat, write the full scene based on the idea. Do not conclude the scene on your own, follow the beat instructions closely. Do not end with foreshadowing.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n- Kontext: Fantasy Roman ANSTURM\n- Charaktere: Raven, Dusk, Caleor, alles Jungs in der 8. Klasse.\n- Perspektive: 1. Person Sicht Caleor, m\u00e4nnlich, Ich-Erz\u00e4hler\n- Zeitform: Gegenwart\n- Raven \u00fcbernimmt die Kontrolle und stellt eine zynische Frage, die die Unlogik der Situation hervorhebt. Er ist frustriert und handelt impulsiv, indem er die Schl\u00fcssel an sich rei\u00dft und die T\u00fcr zuschl\u00e4gt, bevor er entschlossen Richtung B\u00fcro des Schulleiters geht.\n- Die restliche Gruppe bleibt zur\u00fcck. Der Protagonist Caleor lenkt die Aufmerksamkeit au

In [13]:
fine_tuned_scene

'Was sollen wir denn alles tun? Wir werden uns in den Bergen totlaufen, ein paar Stunden lang die Berge erkunden und dann wieder zurückkehren. Das ist doch alles nur ein Vorwand, um eine Biketour zu machen. Das ist doch das Gleiche, wie wenn wir einfach auf dem Radweg entlangfahren würden. Was ist das Besondere an diesem Ort? Was ist so besonders an diesem schönen Ort, den wir letztes Jahr entdeckt haben? Wir werden ihn wieder entdecken, wenn wir ihn erreichen. Das ist doch alles nur ein Haufen Unsinn. Wir sollten uns lieber in den Garten setzen und ein bisschen spielen, statt uns so viel Zeit und Mühe zu geben, um an einen Ort zu gelangen, der uns doch sowieso wieder verlassen wird. Ich werde ihn nicht mehr als ein Tag lang haben, und dann ist er wieder weg. Ich werde ihn nicht mehr als 24 Stunden lang haben, und dann ist er wieder weg. Das ist doch alles nur ein Haufen Unsinn. Wir sollten uns lieber in den Garten setzen und ein bisschen spielen, statt uns so viel Zeit und Mühe zu geb